# Facebook Prophet: Stock Market Prediction

**Objective:** a financial institution aimed to predict stock prices for various companies to optimize trading strategies.

**Approach:** historical stock price data, trading volumes, and financial indicators were used as input. FB Prophet was applied in order to forecast the stock market values.

**Outcome:** the accurate stock price predictions helped the financial institution make better investment decisions and improve trading strategies.